# Loading

Create a generator function for importing the file

In [29]:
import urllib.request
import lzma
import base64
import re

def get_dependencies():
    localfile, info = urllib.request.urlretrieve("https://github.com/kevSweet/meta-deps/raw/master/pypi-deps.csv.lzma")
    with lzma.open(localfile, mode='r') as file:
        for line in file:
            package, version, deps = line.decode("utf-8").split('\t')
            deps = base64.b64decode(deps).decode("utf-8")
            deps = eval(deps)
            regex = r'\"*([\w\.]+)[><]?\=?.*\"*'
            
            def cleanDep(s):
                try:
                    match = re.search(regex, s)
                    return match.group(0)
                except:
                    return None
            
            deps = [cleanDep(x) for x in deps if cleanDep(x) is not None]
            
            yield package, deps
        
        


Store as dataframe

In [30]:
import pandas as pd
import numpy as np

rows = np.array([[None,None]])
for package, deps in get_dependencies():
    # Packages with no dependencies get blank string
    if deps:
        args= [[package, dep] for dep in deps]
    else:
        args = [[package, None]]
    rows = np.append(rows, args, axis=0)

        
df = pd.DataFrame(rows, columns=["package", "dependency"]).drop_duplicates()
df = df.dropna(subset=['package'])
df.head()

,package,dependency
1,0x10c-asm,None
2,2gis,None
3,3to2,None
4,3to2_py3k,None
5,42qucc,None


In [88]:
from timeit import default_timer

## Redshift

In Redshift we create a single table that will hold one "edge" per row.

In [93]:
import redshift_connector
conn = redshift_connector.connect(
     host='redshift-python-dep.cantgzv41ter.us-east-1.redshift.amazonaws.com',
     database='deps',
     user='awsuser',
     password='REDACTED'
  )
conn.autocommit = True
cursor = conn.cursor()

Create the table

In [91]:
cursor.execute(
"""
    CREATE TABLE dependencies (
        package VARCHAR(150),
        dependency VARCHAR(150) NULL
    )
"""
)

ProgrammingError: {'S': 'ERROR', 'C': '42P07', 'M': 'Relation "dependencies" already exists', 'F': '../src/pg/src/backend/tcop/utility.c', 'L': '204', 'R': 'CheckTableNonExistence'}

Load the data

In [ ]:


# Time it
start = default_timer()

cursor.write_dataframe(df, "dependencies")

# for package, deps in get_dependencies():
#     # Packages with no dependencies get blank string
#     if deps:
#         args = list(product([package], deps))
#     else:
#         args = [(package, "")]
        
#     print(args)
    
#     for tup in args:
#         cursor.execute("INSERT INTO dependencies VALUES (%s, %s)", tup)
#     cursor.executemany("INSERT INTO dependencies VALUES (%s, %s)", args)

end = default_timer()
print(end - start) # Time in seconds, e.g. 5.38091952400282

# Neptune Graph

Clear existing data from graph

In [86]:
%%gremlin

g.V().drop()

Load in nodes

In [89]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T

graph = Graph()

remoteConn = DriverRemoteConnection('wss://database-1.cluster-c8vhkkn9knug.us-west-2.neptune.amazonaws.com:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)


for package_name in df.package.unique():
    g.add_v('package').property(T.id, package_name).as_(package_name).next()
    print("added package: " + package_name)

remoteConn.close()

added package: 0x10c-asm
added package: 2gis
added package: 3to2
added package: 3to2_py3k
added package: 42qucc
added package: 4Suite-XML
added package: a8
added package: aaargh
added package: aafigure
added package: aardtools
added package: Aaron
added package: ab
added package: abalone
added package: ABBA
added package: Abe
added package: abifpy
added package: abl.jquery
added package: abl.jquery.ui
added package: abouttag
added package: absolute
added package: absolute32
added package: abu.rpc
added package: AChemKit
added package: acidfs
added package: ackg
added package: acl
added package: acor
added package: acora
added package: acrylamid
added package: actdiag
added package: acted.projects
added package: activecache
added package: ActivityStream
added package: actmon
added package: Adamanteus
added package: ADAM-Tools
added package: addhrefs
added package: AdHoc
added package: adisp
added package: Adjax
added package: adjointShapeOptimizationFlux
added package: adjutant
added pa

In [90]:
%%gremlin

g.V()

Add edges

In [92]:
remoteConn = DriverRemoteConnection('wss://database-1.cluster-c8vhkkn9knug.us-west-2.neptune.amazonaws.com:8182/gremlin','g')
g = graph.traversal().withRemote(remoteConn)


for index, row in df.iterrows():
    print(row['dependency'])
    if row['dependency'] is not None:
        g.V(row['package']).add_e("depends_on").to(row['dependency']).next()
        print("Added dependency: " + row['package'] + "->" + row['dependency'])
        
remoteConn.close()

None
None
None
None
None
None
None
argparse


GremlinServerError: 499: {"requestId":"7fa4f571-8971-4c81-bec8-43d6ed6a5f95","code":"InvalidParameterException","detailedMessage":"The provided traverser does not map to a value: v[aaargh]->[SelectOneStep(last,argparse,null)]"}

In [81]:
%%gremlin

g.E()